### 전략
모델을 세가지로 나누어서 한단계한단계 복원을 진행하도록한다.  

## 모델
1. 마스크생성모델  
  
학습:  
train_gt의 정답이미지를 흑백이미지로 변환하여 train_input의 학습이미지와 비교하여 차이가나는부분을 mask로하고 이것을 새로운 정답이미지로 사용한다.  
이후 train_input파일과 mask이미지를 가지고 손상영역 mask 모델을 학습한다.  
  
테스트:  
손상된부분을 인식하여 mask를 생성하고 각각의 파일을 mask폴더에 저장한다.  
  
2. 컬러복원모델  
  
학습:  
train_gt의 이미지를 mask와 결합하여 손상을 시켜 이것을 새로운 정답이미지로 사용한다.  
train_input파일과 손상된 이미지를 가지고 컬러복원 모델을 학습한다.  
  
테스트:  
test_input파일에서 mask를 적용하여 mask를 제외한 나머지부분의 color를 복원하여 output_grayTocol폴더에 저장한다.
  
3. 손상복원모델  

학습:  
(2)에서 의도하여 손상시킨 손상된 컬러 이미지와 train_gt파일을 가지고 손상복원 모델을 학습한다.  
  
테스트:  
컬러만 복원된 파일에서 mask로 손상된부분을 인식시키고 그 손상된부분을 복원하여 최종 폴더에 저장한다.
단, 컬러복원 모델의 컬러복원성능이 좀더 좋기때문에 이전에 컬러복원된부분은 그대로 사용하도록한다.

----------------------------------------------------------------------------
## 전처리 부가함수
1. 마스크생성기  
  
손상된부분의 마스크를 생성한다.

2. 컬러손상이미지생성기

손상된 컬러이미지를 생성한다.

# 1. 마스크생성모델  
  

  

In [ ]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
from torchvision.models.segmentation import deeplabv3_resnet50, DeepLabV3_ResNet50_Weights
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt

batch = 30
data_ratio = 1 # 100% 데이터셋사용
num_epochs = 60
test_size=0.2
lr=0.001

from sklearn.model_selection import train_test_split

# Dataset and DataLoader
input_dir = '/root/.cache/kagglehub/datasets/geon05/dataset2/versions/1/train_input'
gt_dir = '/root/.cache/kagglehub/datasets/geon05/dataset2/versions/1/train_gt'

# Train-Validation Split (80:20)
image_files = sorted(os.listdir(input_dir))
mask_files = sorted(os.listdir(gt_dir))

# 10% 샘플링된 데이터에서 Train-Validation Split (80:20)
train_images, val_images, train_masks, val_masks = train_test_split(
    image_files, mask_files, test_size=test_size, random_state=42
)

class DamageDataset(Dataset):
    def __init__(self, input_dir, gt_dir, image_files, mask_files, transform=None):
        self.input_dir = input_dir
        self.gt_dir = gt_dir
        self.image_files = image_files
        self.mask_files = mask_files
        self.transform = transform

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        input_path = os.path.join(self.input_dir, self.image_files[idx])
        gt_path = os.path.join(self.gt_dir, self.mask_files[idx])

        # Load and preprocess images
        input_image = Image.open(input_path).convert("RGB")
        input_image_np = np.array(input_image)
        gt_image_gray = Image.open(gt_path).convert("L")
        gt_image_gray_np = np.array(gt_image_gray)

        input_image_gray_np = cv2.cvtColor(input_image_np, cv2.COLOR_RGB2GRAY)
        difference = cv2.absdiff(gt_image_gray_np, input_image_gray_np)
        _, binary_difference = cv2.threshold(difference, 1, 255, cv2.THRESH_BINARY)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 1))
        binary_difference = cv2.morphologyEx(binary_difference, cv2.MORPH_CLOSE, kernel)
        contours, _ = cv2.findContours(binary_difference, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        mask_filled = np.zeros_like(binary_difference)
        cv2.drawContours(mask_filled, contours, -1, color=255, thickness=cv2.FILLED)
        mask_filled = cv2.dilate(mask_filled, kernel, iterations=1)

        input_tensor = transforms.ToTensor()(input_image)
        mask_tensor = torch.tensor(mask_filled, dtype=torch.float32).unsqueeze(0) / 255.0

        return input_tensor, mask_tensor

# Training and Validation Datasets
train_dataset = DamageDataset(input_dir, gt_dir, train_images, train_masks)
val_dataset = DamageDataset(input_dir, gt_dir, val_images, val_masks)

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False)

# Load model with updated weights parameter
weights = DeepLabV3_ResNet50_Weights.DEFAULT
model = deeplabv3_resnet50(weights=weights)
model.classifier[4] = nn.Conv2d(256, 1, kernel_size=1)

# GPU 사용 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")  # 주 GPU로 cuda:0을 사용하도록 설정

# DataParallel 및 SyncBatchNorm 적용
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs! Model training started...")
    model = nn.DataParallel(model)  # DataParallel로 모델 병렬화
    model = nn.SyncBatchNorm.convert_sync_batchnorm(model)  # SyncBatchNorm 변환

# GPU로 모델 이동
model = model.to(device)

# Loss and Optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training and Validation Loop

best_val_loss = float('inf')

for epoch in range(num_epochs):
    # Training Phase
    model.train()
    train_loss = 0.0
    for inputs, masks in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Training"):
        inputs, masks = inputs.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)['out']
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    avg_train_loss = train_loss / len(train_loader)

    # Validation Phase
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for inputs, masks in tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} - Validation"):
            inputs, masks = inputs.to(device), masks.to(device)
            outputs = model(inputs)['out']
            loss = criterion(outputs, masks)
            val_loss += loss.item()
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"  Training Loss: {avg_train_loss:.4f}")
    print(f"  Validation Loss: {avg_val_loss:.4f}")

    # Save best model
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        torch.save(model, f"best_model_{epoch+1}_{best_val_loss:.4f}.pth")
        print(f"  Best model saved with Validation Loss: {best_val_loss:.4f}")

    # Visualization (Optional)
    with torch.no_grad():
        inputs, masks = next(iter(val_loader))
        inputs, masks = inputs[:5].to(device), masks[:5].to(device)
        predictions = torch.sigmoid(model(inputs)['out'])
        predictions = predictions.cpu().numpy()
        masks = masks.cpu().numpy()
        inputs = inputs.cpu().numpy()

        fig, axes = plt.subplots(5, 3, figsize=(12, 15))
        for i in range(5):
            axes[i, 0].imshow(inputs[i].transpose(1, 2, 0))
            axes[i, 0].set_title("Input Image")
            axes[i, 0].axis("off")
            axes[i, 1].imshow(masks[i][0], cmap="gray")
            axes[i, 1].set_title("Ground Truth Mask")
            axes[i, 1].axis("off")
            axes[i, 2].imshow(predictions[i][0], cmap="gray")
            axes[i, 2].set_title("Predicted Mask")
            axes[i, 2].axis("off")
        plt.tight_layout()
        plt.show()
